import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
import string
from dateutil import parser

In [279]:
def get_soup(start_url):
    response = requests.get(start_url)
    page = response.text
    return BeautifulSoup(page, "lxml")

In [280]:
from datetime import date, datetime, timedelta

def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

urls = []
for suffix in perdelta(date(2019, 8, 7), date(2019, 11, 30), timedelta(days=30)):
    prefix = 'http://www.billboard.com/charts/hot-100/'
    link = prefix + str(suffix)
    urls.append(link)

In [281]:
def get_artist(soup):
    artist = soup.find_all(class_="chart-element__information__song text--truncate color--primary")
    artists = []
    for item in artist:
        item = item.text.strip()
        artists.append(item)
    return artists

In [282]:
def get_track(soup):
    track = soup.find_all(class_="chart-element__information__artist text--truncate color--secondary")
    tracks = []
    for item in track:
        item = item.text.strip()
        tracks.append(item)
    return tracks



In [272]:
frames = []
billboard_dict = {}
for url in urls:
    soup = get_soup(url)
    artist = get_artist(soup)
    track = get_track(soup)
    dates = [parser.parse(url.split('/')[5])]*len(artist)
    df = pd.DataFrame({'artist': artist, 'track': track, 'publish_date': dates})
    frames.append(df)

In [283]:
df_merge = pd.concat(frames).reset_index(drop=True)

In [274]:
df_merge.to_csv('File1.csv')

In [2]:
df2 = pd.read_csv('File2.csv')
print(df2.shape)
df3 = df2.drop_duplicates(['track'], keep='first').reset_index(drop=True)

(36500, 4)


In [3]:
print(df3.shape)
df3.to_csv('File3.csv')

(8755, 4)
